### Gradient approach for PCA

### Load libraries

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from numpy.linalg import eig
import matplotlib.pyplot as plt
from numpy import linalg as LA

C:\ProgramData\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


### Loading iris data

In [2]:
iris_d = sm.datasets.get_rdataset('iris')
iris = iris_d.data
iris.rename(columns=lambda x: x.replace('.', ''), inplace=True)
iris.head()

C:\ProgramData\Anaconda\lib\site-packages\statsmodels\datasets\utils.py:192: FutureWarning: `item` has been deprecated and will be removed in a future version
  return dataset_meta["Title"].item()


,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
df_iris=iris.drop(labels='Species', axis=1)

### Scaling the data

In [4]:
sc = StandardScaler()
sc.fit(df_iris)
df_std = pd.DataFrame(sc.transform(df_iris),columns={'SepalLength','SepalWidth','PetalLength','PetalWidth'})
df_std.head()

,PetalWidth,PetalLength,SepalLength,SepalWidth
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444


### Compute the varaince-covariance matrix

In [5]:
V = np.cov(df_std.T)

### Computing eigen values and eigen vectors

In [6]:
val, vec = eig(V)
print('eigen values:',val)
print('Eigen vactors:')
vec

eigen values: [2.93808505 0.9201649  0.14774182 0.02085386]
Eigen vactors:


array([[ 0.52106591, -0.37741762, -0.71956635,  0.26128628],
       [-0.26934744, -0.92329566,  0.24438178, -0.12350962],
       [ 0.5804131 , -0.02449161,  0.14212637, -0.80144925],
       [ 0.56485654, -0.06694199,  0.63427274,  0.52359713]])

### The variance each component explains

In [7]:
exp_var = val / np.sum(val)
print('Explained variances:', exp_var)

Explained variances: [0.72962445 0.22850762 0.03668922 0.00517871]


### PC1

In [8]:
PC1 = df_std.dot(vec[:,0])
PC1.head()


0   -2.264703
1   -2.080961
2   -2.364229
3   -2.299384
4   -2.389842
dtype: float64

### PC2

In [9]:
PC2 = df_std.dot(vec[:,1])
PC2.head()

0   -0.480027
1    0.674134
2    0.341908
3    0.597395
4   -0.646835
dtype: float64

### Another way to write the code

We try to minimize the Lagrange function to find the eigen vector($a_0$).
However, in lagrange formula we have two unknowns (eigen value and eigen vector) and I tried to write eigen value as ${a_0}^T*\sum*a_0$ and estimate the eigen vector but the gradient descent approach does not converge to the eigen vectors that numpy linear algebra package gives us.
The critical points of Lagrangians occur at saddle points, rather than the local maxima or minima. And numerical optimization technique such as gradient descent/ascent are not designed to find the saddle point. That is the reason my code does not converge.

### PC1

This is Covariance-free computation of first PC

In [10]:
r = np.ones(4)*0.5
r = r/LA.norm(r)
x = np.array(df_std)
num_itr = 100
for c in range(num_itr):
    s = np.zeros(len(r))
    for j in range(len(x)):
        s = s + (x[j]@r)*x[j]
    eigenV = r.T@s
    err = abs(eigenV*r-s)
    r = s/LA.norm(s)

#### First component eigen vector

In [11]:
r

array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654])

#### PC1 by this method

In [12]:
PC1 = df_std.dot(r)
PC1.head()

0   -2.264703
1   -2.080961
2   -2.364229
3   -2.299384
4   -2.389842
dtype: float64

### Gradient approach for PC1 (finding optimum from the Lagrange function)

In [13]:
a1 = np.ones(4)*0.5
eps = 0.000001
L =1
i=0
while abs(L) > eps:
    lmbda = a1.T@V@a1
    L = a1.T@V@a1 - lmbda*(a1.T@a1-1)
    a = a1-2*(V@a1-lmbda*a1)
    a1 = a
    i+=1

a1

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in double_scalars
  import sys
C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in multiply
  
C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in matmul
  
C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in matmul
  import sys
C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in matmul
  


array([nan, nan, nan, nan])

#### First component eigen vector

In [14]:
a1

array([nan, nan, nan, nan])

#### PC1 by this method

In [15]:
PC1 = df_std.dot(a1)
PC1.head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64

### Gradient approach for PC2 (finding optimum from the Lagrange function)

We can apply the same approach for the second PC with different Lagrange function

In [16]:

a1 = vec[:,0]
lmbda = val[0]
a = np.ones(4)*0.5
a2 = np.ones(4)
eps = 0.0001
i=0
L=1
while abs(L) > eps:
    alpha = 2*a1.T@V@a2
    L = a2.T@V@a2 - lmbda*(a2.T@a2-1) - alpha*(a2.T@a1)
    a = a2 - (2*(V-lmbda*np.eye(4))@a2 - alpha*a1)
    a2 = a
    i+=1 

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in matmul
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


#### Second component eigen vector

In [17]:
a2

array([ 5.80798991e+154, -3.00659635e+154,  6.87297544e+154,
        6.20850157e+154])

#### PC2 by this method

In [18]:
PC2 = df_std.dot(a2)
PC2.head()

0   -2.567316e+155
1   -2.362010e+155
2   -2.680245e+155
3   -2.603280e+155
4   -2.706901e+155
dtype: float64